#PLC数据，添加一些新的feature

#运动方向：x,y,z
#方向是否改变: 1/0
#位移量：x,y,z

#load改变量

#todo:后续聚集成为1min的一些特征


In [3]:
#探索PLC文件不同分钟的细节

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

import pandas as pd
import matplotlib.pyplot as plt

import os.path
import datetime
from datetime import datetime
#训练文件夹
TrainFolder='H:\\12_Kaggle\\toollife\\03-FinalData-r1el'
# TrainFolder='H:\\12_Kaggle\\toollife\\02-TestingData-poL3'



#处理后保存文件
dstPath='H:\\12_Kaggle\\toollife\\pythonProject\\FinalPreProcessed'

In [6]:
TrainFileList=[]
ResultFileList=[]
for n in range(4):
    pathend='0'+str(n+1)+"\\PLC\\plc.csv"
    folder=os.path.join(TrainFolder,pathend)
    TrainFileList.append(folder)
    
    resultend=str(n+1)+".csv"
    resultfolder=os.path.join(dstPath,resultend)
    ResultFileList.append(resultfolder)

# TrainFolder01=os.path.join(TrainFolder,"01\\PLC")
# TrainFolder02=os.path.join(TrainFolder,"02\\PLC")
# TrainFolder03=os.path.join(TrainFolder,"03\\PLC")
# TrainFolder03=os.path.join(TrainFolder,"04\\PLC")
# TrainFolder03=os.path.join(TrainFolder,"05\\PLC")
# #第一个PLC文件
# Train01Plc=os.path.join(TrainFolder01,'plc.csv')
# Train02Plc=os.path.join(TrainFolder02,'plc.csv')
# Train03Plc=os.path.join(TrainFolder03,'plc.csv')
TrainFileList,ResultFileList

(['H:\\12_Kaggle\\toollife\\03-FinalData-r1el\\01\\PLC\\plc.csv',
  'H:\\12_Kaggle\\toollife\\03-FinalData-r1el\\02\\PLC\\plc.csv',
  'H:\\12_Kaggle\\toollife\\03-FinalData-r1el\\03\\PLC\\plc.csv',
  'H:\\12_Kaggle\\toollife\\03-FinalData-r1el\\04\\PLC\\plc.csv'],
 ['H:\\12_Kaggle\\toollife\\pythonProject\\FinalPreProcessed\\1.csv',
  'H:\\12_Kaggle\\toollife\\pythonProject\\FinalPreProcessed\\2.csv',
  'H:\\12_Kaggle\\toollife\\pythonProject\\FinalPreProcessed\\3.csv',
  'H:\\12_Kaggle\\toollife\\pythonProject\\FinalPreProcessed\\4.csv'])

In [102]:
#读入PLC文件，03共85293行


In [7]:
#新建列

df_file_list=[]
fileindex=0
for file in TrainFileList:
    #读入文件
    df_all=pd.read_csv(file)
    #添加列
    df_all.loc[:,'x_displacement']=0
    df_all.loc[:,'y_displacement']=0
    df_all.loc[:,'z_displacement']=0
    df_all.loc[:,'displacement']=0
    df_all.loc[:,'x_move']=0  #是否在这个方向上移动，0或者1
    df_all.loc[:,'y_move']=0
    df_all.loc[:,'z_move']=0
    df_all.head()

    
    headlist=df_all.columns
    headlist=list(headlist)
    
    #每一个df分别是1分钟
    #需要将位置数据转化为速度数据，x,y,z方向的数据
    #需要给整个df每一行添加一些新的feature，方向是否改变【0,1】，方向改变【0,x,y,z,xy,xz,yz,xyz...】
    
    #建立一个dflist，分别是对应每个csvno
    dflist=[]
    for i in range(1,49):
        dfi=df_all[df_all.csv_no == i]
        dflist.append(dfi)
        
    #遍历计算，再重新拼合保存
    df_list2=[]
    df_index=0
    for df in dflist:
    
        #转换为np格式运算
        da=np.array(df)
        print('processing ',df_index)

        for i in range(1,len(da)-1): #从每个df的第二行开始，循环每一行
    #         print(i)
            q=i-1
        #     if da[i].all==da[q].all: continue

            da[i][6]=da[i][2]-da[q][2] #x
            da[i][7]=da[i][3]-da[q][3] #y
            da[i][8]=da[i][4]-da[q][4] #z
            da[i][9]=np.sqrt(da[i][6]**2+(da[i][7])**2+(da[i][8])**2)

            if da[i][6]!=0: da[i][10]=1
            if da[i][7]!=0: da[i][11]=1    
            if da[i][8]!=0: da[i][12]=1

        df_index+=1

        df2=pd.DataFrame(da)
        
        #添加机器号标记
        

        df_list2.append(df2)
        
    #拼合

    df_processed=pd.concat(df_list2)
    df_processed.columns=headlist
        
        #保存下来
#         df_file_list.append(df_processed)

    #直接保存为文件
    df_processed.to_csv(ResultFileList[fileindex])
    fileindex+=1
        
        


processing  0
processing  1
processing  2
processing  3
processing  4
processing  5
processing  6
processing  7
processing  8
processing  9
processing  10
processing  11
processing  12
processing  13
processing  14
processing  15
processing  16
processing  17
processing  18
processing  19
processing  20
processing  21
processing  22
processing  23
processing  24
processing  25
processing  26
processing  27
processing  28
processing  29
processing  30
processing  31
processing  32
processing  33
processing  34
processing  35
processing  36
processing  37
processing  38
processing  39
processing  40
processing  41
processing  42
processing  43
processing  44
processing  45
processing  46
processing  47
processing  0
processing  1
processing  2
processing  3
processing  4
processing  5
processing  6
processing  7
processing  8
processing  9
processing  10
processing  11
processing  12
processing  13
processing  14
processing  15
processing  16
processing  17
processing  18
processing  19


In [20]:
df_all_files=pd.concat(df_file_list)
df_all_files.to_csv(dstPath)

In [21]:
df_all_files.head()

,time,spindle_load,x,y,z,csv_no,x_displacement,y_displacement,z_displacement,displacement,x_move,y_move,z_move
0,13:45:00:13,0.064089,-636.564,46.7288,-380.029,1,0,0,0,0,0,0,0
1,13:45:00:29,0.064089,-636.564,46.7288,-380.029,1,0,0,0,0,0,0,0
2,13:45:00:44,0.064089,-636.564,46.7288,-380.029,1,0,0,0,0,0,0,0
3,13:45:00:60,0.064089,-636.564,46.7288,-380.029,1,0,0,0,0,0,0,0
4,13:45:00:75,0.064089,-636.564,46.7288,-380.029,1,0,0,0,0,0,0,0


In [1]:
datetime.strptime(t,'%H:%M:%S:.%f')
date_2017 = datetime(year=2017, month=12, day=31, hour=11, minute=23, second=12,microsecond=1)

NameError: name 'datetime' is not defined

In [119]:
t=df.iloc[0,0]

# 拼合3个Summary在一起


In [8]:
folder='H:\\12_Kaggle\\toollife\\pythonProject\\FinalPreProcessed'
dst=os.path.join(folder,'PLC_All.csv')

In [9]:
os.listdir(folder)

['1.csv', '2.csv', '3.csv', '4.csv']

In [10]:
dfs=[]
findex=0
for x in os.listdir(folder):
    path=os.path.join(folder,x)
    
    df=pd.read_csv(path)
    df['Machine']=findex
    dfs.append(df)
    findex+=1

In [11]:
dfnew=pd.concat(dfs)

In [12]:
dfnew

,Unnamed: 0,time,spindle_load,x,y,z,csv_no,x_displacement,y_displacement,z_displacement,displacement,x_move,y_move,z_move,Machine
0,0,00:00:00.000,13.898129,-753.271118,409.624512,-380.278015,1,0.000000,0.000000,0.0,0.000000,0,0,0,0
1,1,00:00:00.010,13.898129,-753.271118,409.624512,-380.278015,1,0.000000,0.000000,0.0,0.000000,0,0,0,0
2,2,00:00:00.420,22.367016,-754.035400,348.161499,-380.278015,1,-0.764282,-61.463013,0.0,61.467765,1,1,0,0
3,3,00:00:00.430,22.367016,-754.035400,348.161499,-380.278015,1,0.000000,0.000000,0.0,0.000000,0,0,0,0
4,4,00:00:00.440,22.367016,-754.035400,348.161499,-380.278015,1,0.000000,0.000000,0.0,0.000000,0,0,0,0
5,5,00:00:00.450,22.367016,-754.035400,348.161499,-380.278015,1,0.000000,0.000000,0.0,0.000000,0,0,0,0
6,6,00:00:00.460,22.367016,-754.035400,348.161499,-380.278015,1,0.000000,0.000000,0.0,0.000000,0,0,0,0
7,7,00:00:00.470,22.367016,-754.035400,348.161499,-380.278015,1,0.000000,0.000000,0.0,0.000000,0,0,0,0
8,8,00:00:00.480,22.367016,-754.035400,348.161499,-380.278015,1,0.000000,0.000000,0.0,0.000000,0,0,0,0
9,9,00:00:00.490,22.367016,-754.035400,348.161499,-380.278015,1,0.000000,0.000000,0.0,0.000000,0,0,0,0


In [13]:
dfnew.to_csv(dst)